In [ ]:
filename = r'data\022rde11_T_minus15_acc__NT_plus10_don_1.sm'

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import phconvert as phc
phc.__version__

# Author

In [ ]:
author = 'Eitan Lerner'
author_affiliation = 'UCLA'
creator = 'Antonino Ingargiola'
creator_affiliation = 'UCLA'

# Sample

In [ ]:
description = 'A demostrative smFRET-usALEX measurement.'
sample_name = '022rde11_T_minus15_acc__NT_plus10_don_1'
dye_names = ['ATTO550', 'ATTO647N']
buffer_name = 'Transcription buffer.'

# Prepare data

## Read the data

In [ ]:
d = phc.loader.usalex_sm(filename,
                         donor = 0,
                         acceptor = 1,
                         alex_period = 4000,
                         alex_offset = 700,
                         alex_period_donor = (2180, 3900),
                         alex_period_acceptor = (200, 1800),
                         excitation_wavelengths = (532e-9, 635e-9),
                         detection_wavelengths = (580e-9, 680e-9))

In [ ]:
phc.plotter.alternation_hist(d)

## Add author and sample info

In [ ]:
d['description'] = description

d['sample'] = dict(
    sample_name=sample_name,
    dye_names=[n.encode() for n in dye_names],
    buffer_name=buffer_name,
    num_dyes = len(dye_names))

d['identity'] = dict(
    author=author,
    author_affiliation=author_affiliation,
    creator=creator,
    creator_affiliation=creator_affiliation)

# Validate the Photon-HDF5 structure

Before writing to disk, we assure the file structure follows the Photon-HDF5 format:

In [ ]:
phc.hdf5.assert_valid_photon_hdf5(d)

# Save to Photon-HDF5

In [ ]:
phc.hdf5.save_photon_hdf5(d)

In [ ]:
d['_data_file'].close()

# Load Photon-HDF5

In [ ]:
from pprint import pprint

In [ ]:
filename = d['_data_file'].filename

In [ ]:
h5data = phc.hdf5.load_photon_hdf5(filename)

In [ ]:
phc.hdf5.dict_from_group(h5data.identity)

In [ ]:
phc.hdf5.dict_from_group(h5data.setup)

In [ ]:
pprint(phc.hdf5.dict_from_group(h5data.photon_data))